## Plain Language Summary

A series of phenomena such as early flowering of plants and
early migratory birds are suggesting that the traditional four seasons may have changed. We focus on how
the four seasons changed during 1952–2011 and will change by the end of this century in the warming
Northern Hemisphere midlatitudes. We find that lengths and start dates of the four seasons have changed,
and the changes will be amplified in the future. Over the period of 1952–2011, <i><u>the length of summer
increased from 78 to 95 days and that of spring, autumn and winter decreased from 124 to 115, 87 to 82,
and 76 to 73 days, respectively. In addition, summer is projected to last nearly half a year, but winter less
than 2 months by 2100</u></i>. Such changes can trigger a chain of reactions in agriculture, policy-making for
agricultural management and disaster prevention requires adjustment accordingly. The seasonal-related
topics involving ecology, the ocean and the atmosphere also need to be revisited.

#### Here, we consider how the four seasons changed during 1952–2011 and will change by the end of this century in the warming Northern Hemisphere midlatitudes (30–60°N), where the division of the four seasons based on temperature is readily available.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import geopandas as gpd

In [3]:
summ = 21
wint = 12
aut = 19
spring = 15

In [4]:
def returnLen(file, y1,y2):
    
    def checkLeap(x):
        if (x % 4) == 0:
            if (x % 100) == 0:
                if (x % 400) == 0:
                    return True
                else:
                    return False
            else:
                return True
        else:
            return False
    
    years = [x for x in range(y1,y2)]
    days = [366 if x == True else 365 for x in years]
    
    sum_li = []
    wint_li = []
    spring_li = []
    autumn_li = []
    years_li = []
    location = []
    for x in list(set(file['lat'])):
        for y in list(set(file['lon'])):
            temp = file[(file['lat'] == x)&(file['lon'] == y)]['tavg']
            start = 0
            end = 0
            for d in range(len(days)):
                summer = 0
                winter = 0
                spring = 0
                autumn = 0
                end += days[d]
                flag = 0
                for t in temp[start:end:1]:
                    if (t < spring) and (flag == 0):
                        winter += 1
                    elif (t >= spring) and (t < summ):
                        spring += 1
                    elif (t >= summ):
                        summer += 1
                        flag = 1
                    elif (t <= aut) and (t > wint):
                        autumn += 1
                    elif (t <= wint):
                        winter += 1
                        flag = 0
                start = end
                sum_li.append(summer)
                wint_li.append(winter)
                spring_li.append(spring)
                autumn_li.append(autumn)
                location.append([x,y])
                years_li.append(years[d])
    store = pd.DataFrame()
    store['Year'] = years_li
    store['Location'] = location
    store['Spring'] = spring_li
    store['Winter'] = wint_li
    store['Autumn'] = autumn_li
    store['Summer'] = sum_li
    return store

In [7]:
def preProcess(ds):
    lat = []
    lon = []
    file = pd.DataFrame()
    for t in range(4018):
        for x in ds['latitude'][12:25]:
            for y in ds['longitude'][:]:
                lat.append(x)
                lon.append(y)        
    file['lat'] = lat
    file['lon'] = lon
    li = []
    for i in range(4018):
        for j in range(96):
            li.extend(list((ds['tmin'][i,12:25,j] +  ds['tmax'][i,12:25,j])/2))
    file['tavg'] = li
    file['lon'] = file['lon'].apply(lambda x: x - 360 if x > 180 else x)
    file = file.dropna()
    file = file.reset_index(drop=True)
    return file

In [8]:
ds = nc.Dataset("D:\\Earth.Org\\4seasons\\data\\HadGHCND_TXTN_acts_1950-2014_15102015\\HadGHCND_TXTN_acts_1950-1960_15102015.nc")
#temp = preProcess(ds)
store = returnLen(preProcess(ds), 1950, 1960)

KeyboardInterrupt: 

In [ ]:
years = [1949,1960, 1970, 1980,1990,2000,2010,2014]
for i in range(len(years)-1):
   ds = nc.Dataset(f"D:\\Earth.Org\\4seasons\\data\\HadGHCND_TXTN_acts_1950-2014_15102015\\HadGHCND_TXTN_acts_{years[i]+1}-{years[i]}_15102015.nc") 